# Baseline 1.0

The plan is to build a WavLM_Base + ECAPA-TDNN model mainly to understand how the whole thing works. 

In [1]:
import torchaudio

## WavLM_Base

- [docs](https://pytorch.org/audio/main/generated/torchaudio.pipelines.WAVLM_BASE.html)
- [source](https://github.com/microsoft/unilm/tree/65f15af2a307ebb64cfb25adf54375b002e6fe8d/wavlm#pre-trained-models)
- MIT license
- pretrained on 960h of Librispeech @ 16kHz, make sure that your speech input is also sampled at 16kHz

In [2]:
bundle = torchaudio.pipelines.WAVLM_BASE

In [4]:
# Construct the model and load the pretrained weight
model = bundle.get_model()

Downloading: "https://download.pytorch.org/torchaudio/models/wavlm_base.pth" to /home/milan/.cache/torch/hub/checkpoints/wavlm_base.pth
6.5%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

16.7%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

33.0%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

C

In [5]:
model

Wav2Vec2Model(
  (feature_extractor): FeatureExtractor(
    (conv_layers): ModuleList(
      (0): ConvLayerBlock(
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
      )
      (1-4): 4 x ConvLayerBlock(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      )
      (5-6): 2 x ConvLayerBlock(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
      )
    )
  )
  (encoder): Encoder(
    (feature_projection): FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (pos_conv_embed): ConvolutionalPositionalEmbedding(
        (conv): ParametrizedConv1d(
          768, 768, kernel_size=(128,), stride=(1,), padding=(64,), groups=16
          (parametriza

## ECAPA-TDNN

you need to feed into it the weighted average of all layers

In [ ]:
# https://github.com/TaoRuijie/ECAPA-TDNN/blob/4904fda6c5da208998321e3ea75e24320bab8aad/model.py#L132C1-L132C6
from ecapa_tdnn import ECAPA_TDNN
# TODO look at layers after ECAPA_TDNN encoder

## Loading the dataset

Make sure it's 16MHz
waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

I can start training on Voxceleb1 & 2

Besides, we also did data augmentation
for the training data using the MUSAN [35] noise and RIR 1 rever-
beration with probability 0.6 in online mode.

Same as the implementation in [1], we also use the additive an-
gular margin (AAM) [33] loss in the training process for model op-
timization.
AAM-softmax loss,
margin 0.2

https://paperswithcode.com/method/arcface

ization.
The training pipeline is mainly divided into two stages. In
the first stage, the pre-trained model is fixed. We only update the
ECAPA-TDNN and the weight w for all the hidden states. Then,
we fine-tune all the parameters for pre-trained model and ECAPA-
TDNN

During the training process,
we randomly sampled 3s segment from each utterance to construct
training batch. For the two-stage training pipeline described in sec-
tion 3.2.2, we first fixed the pre-trained model and trained for 10
epochs. Then, we fine-tuned all the parameters for another 5 epochs.
Besides, to further improve our best system, we did large margin
fine-tuning [36] by randomly sampling 6s segments and set the AAM
margin to 0.5 to train extra 2 epochs.

During the evaluation, we use the cosine score to measure the
similarity for trial pairs. We also use the adaptive s-norm [37, 38] to
normalize the scores in our experiment. The embeddings extracted
from the training set are averaged according to the speaker label and
used as the imposter cohort. We set the imposter cohort size to 600
in our experiment. When doing quality-aware score calibration [36],
we randomly generated 30k trials based on the voxceleb2 test set to
train our calibration model.

This is good: https://github.com/microsoft/unilm/tree/65f15af2a307ebb64cfb25adf54375b002e6fe8d/wavlm#pre-trained-models
